In [ ]:
'''random forest binary classifier - ready'''

#import pandas for data processing
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
def fname_ls_builder(fin_path):
  #Find all files in data directory
  from os import listdir
  from os.path import isfile, join
  return [f for f in listdir(fin_path) if isfile(join(fin_path,f))]

def df_builder(fin_path,fname_ls):
  #this function imports data from CSV files to one large dataframe

  #create list to hold dataframes
  df_ls=[]

  #read in each file
  for i in fname_ls:
    temp_df=pd.read_csv(fin_path+i,index_col=0)
    #print(i,'loaded\n\n')
    df_ls.append(temp_df)

  #create one large df
  df=pd.concat(df_ls)
  df.dropna(inplace=True)
  print(df.shape)
  return df

def split_data(X):
  #separate y from X
  y=X[['label']]
  X.drop(['label'],axis=1,inplace=True)
  return X,y

def get_data(pkin_path):
  #create list of files, from that build the dataframe
  pk_fname_ls=fname_ls_builder(pkin_path)
  pk_df=df_builder(pkin_path,pk_fname_ls)

  #drop relative intensities
  pk_df.drop(columns=[i for i in pk_df.columns.values if 'val' in i],inplace=True)
  return pk_df


In [ ]:
def scale_data(pk_df):
  #scale data((X-mean)/std_dev)
  from sklearn.preprocessing import StandardScaler
  sc = StandardScaler()
  X = sc.fit_transform(pk_df)
  return X

In [ ]:
def set_split(target_set,noise_set):
  #get the size of the noise_set for use in the model
  size=int((target_set.shape[0]/noise_set.shape[0])*1.2*noise_set.shape[0])

  #randomly select 1.2x the target_set rows from the noise set 
  from numpy.random import default_rng
  msk = default_rng(seed=0).uniform(0,1,len(noise_set)) < size
  noise_set = noise_set[msk]
  noise_set.label=0
  target_set.label=1
  X,y=split_data(noise_set.append(target_set))

  #split into train and dev sets
  from sklearn.model_selection import train_test_split
  return train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
def train_model(X_train,y_train):
  #create model
  from sklearn.ensemble import RandomForestClassifier
  return RandomForestClassifier(n_estimators=100).fit(X_train,y_train)


In [ ]:
def assess_model(model,X_dev,y_dev,m_label):
  #check model
  print(model.score(X_dev,y_dev))

  from sklearn.metrics import confusion_matrix
  cm=confusion_matrix(y_dev,model.predict(X_dev))

  print('Model label ',m_label,'\n',cm,'\n\n\n')
  print('Model prectictions',model.predict_proba(X_dev))

In [ ]:
def train_models():
  #set filepaths for import/export
  pkin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Labeled/'

  #build master dataframe of data from pkin_path
  df=get_data(pkin_path)

  #list for holding models
  m_list=[]
  l_list=sorted(df['label'].unique().tolist())
  #loop to create one model for each label in df
  for i in l_list:
    print('in loop, label is',i,'\n\n')
    #for label i, select the target label set and create a training and dev set
    target_df=df.loc[df['label']==i,:]
    noise_df=df.loc[df['label']!=i,:]
    X_train,X_dev,y_train,y_dev=set_split(target_df,noise_df)
    #print('\ndata split\n',X_train.head(),'\n\n',y_train.head())
    m_list.append(train_model(X_train,y_train.values.ravel()))
    print('model trained')
    assess_model(m_list[-1],X_dev,y_dev,i)
  return m_list,l_list

  

In [ ]:
def main():
  #train the models for testing
  m_list,l_list=train_models()
  #set path for test data
  test_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Unlabeled/'
  test_df=get_data(test_path)
  print(len(test_df.index),",",len(m_list))
  labels=np.empty(shape=(len(test_df.index),len(m_list)))
  for j in range(len(m_list)):
    labels[:,j]=m_list[j].predict_proba(test_df)[:,1]
  label_df=pd.DataFrame(labels,index=test_df.index.values,columns=l_list)
  print(label_df)
  label_df.to_csv('/content/drive/My Drive/ML Spectroscopy/Programs/Data Processing/Saved Lists/binary_classifier_labels.csv')

main()

(286406, 11)
in loop, label is 0 


model trained
0.999598477706784
Model label  0 
 [[45941     4]
 [   19 11318]] 



Model prectictions [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
in loop, label is 1 


model trained
0.9999650850179812
Model label  1 
 [[52038     1]
 [    1  5242]] 



Model prectictions [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
in loop, label is 2 


model trained
0.9997032226528403
Model label  2 
 [[51178     2]
 [   15  6087]] 



Model prectictions [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
in loop, label is 3 


model trained
0.9998777975629343
Model label  3 
 [[50160     0]
 [    7  7115]] 



Model prectictions [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
in loop, label is 4 


model trained
1.0
Model label  4 
 [[55203     0]
 [    0  2079]] 



Model prectictions [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
in loop, label is 5 


model trained
0.9999127125449531
Model label 